<table width=100%>
<tr>
    <td><h1 style="text-align: left; font-size:300%;">
        Homework-1
    </h1></td>
    <td width="20%">
    <div style="text-align: right">
    <b> ANN2DL 2022</b> <br>
    <b> First Homework - 09/11/2022<br>
    <b> Team: Heart of Gold<br>
    <b> Components:<br>
    Emanuele Bianchi <br>
    <a href="mailto:emanuele6.bianchi@mail.polimi.it">emanuele.bianchi@mail.polimi.it</a><br>
    Junchong Huang <br>
    <a href="mailto:junchong.huang@mail.polimi.it">junchong.huang@mail.polimi.it</a><br>
    Carlo Manenti <br>
    <a href="mailto:carlo.manenti@mail.polimi.it">carlo.manenti@mail.polimi.it</a><br>
    </div>
    </td>

</tr>
</table>

# Define the enviorement

DATASET PATH (DOWN HERE)

In [ ]:
DATASET_PATH = 'inser here the path to the training data final'
VALIDATION_PATH = 'inser here a path, in it it will be defined a folder with the validation data in'
TRAINING_PATH = 'inser here a path, in it it will be defined a folder with the training data in'

Importing libraries

In [ ]:
# general imports
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# suppressing warnings
import warnings
import logging

# plots
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix


# manual split and augmentations
from PIL import Image
import shutil
import albumentations as A
import cv2
from pathlib import Path
from tqdm import tqdm

# nice videos 
from IPython.display import YouTubeVideo

# gettin informations about tensorflow (different versions may lead to slightly different results)
tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

note: our version of tensor flow is 2.6.4 since we are using kaggle notebooks to run our scripts. Using Colab will gave us access to a much more up to date version of Tensorflow, but will also render the training step far to slow! Another reason we picked kaggle over Colab is the defined time limit for GPU acceleration. Kaggle give us 30h per week, while Colab does not give us clear information about what we are using as accelators.

Planting a seed for reproducibility. Our seed of choice is of course 42! 
note: a seed helps to make our notebook more reproducible, but training model with GPU accelarations will lead to some small variances. Mainly beacuse GPUs leverages stochastics process which renders impossible to repuce exactly the results.  

In [ ]:
seed = 42 # 'So long and thanks for all the fish' 

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

Optional: suppres warnings for a cleaner code

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Understanding the data

## Loading the data

Defining the labels

In [ ]:
# writing labels
labels = {0 : 'Species1',             # 0
          1 : 'Species2',             # 1
          2 : 'Species3',             # 2
          3 : 'Species4',             # 3
          4 : 'Species5',             # 4
          5 : 'Species6',             # 5
          6 : 'Species7',             # 6
          7 : 'Species8'}             # 7

Defining a generator to access the data

In [ ]:

# defining the arguments
target_size = (96, 96)
color_mode = 'rgb'
classes = None
class_mode = 'categorical'
batch_size = 64

#loading the data
dataset_dir = Image.Path(DATASET_PATH)
data_gen = ImageDataGenerator()
train_gen = data_gen.flow_from_directory(directory = dataset_dir,
                                               target_size = target_size,
                                               color_mode = color_mode, 
                                               class_mode = class_mode,
                                               classes = classes,
                                               batch_size = batch_size,
                                               shuffle=False,
                                               seed=seed)

We have a total of 3542 image and 8 different classes

Let's have a look at how data is organised

In [ ]:
print("Depth 1 (traning data final")
print("-------")
#!ls /kaggle/input/homework-1/training_data_final

In [ ]:
print("Depth 2 (Species 1")
print("-------")
#!ls /kaggle/input/homework-1/training_data_final/Species1

Each species has its own subfolder with its images. This means that we must decide how to split our data into train and validation sets.
But first it is better to have a look at some of the images to get an idea of what we are working with.

## Looking at the data

In [ ]:
# can take a while to load, help yourself with something to drink or eat. 
num_row = 3
num_col = 5
fig, axes = plt.subplots(num_row, num_col, figsize=(5*num_row,2*num_col))
for i in range(num_row*num_col):
    batch = next(train_gen)
    image = batch[0]
    image = image[0]
    
    target = batch[1]
    target = target[0]
    target_idx = np.argmax(target)
    
    ax = axes[i//num_col, i%num_col]
    ax.imshow(np.uint8(image))
    ax.set_title('{}'.format(labels[target_idx]))

 We can start to get an idea of the problem!

A very poor way of loading the data starting from the generators

In [ ]:
# loading all the data
train_gen.reset() # resetting to the first batch
X_train, y_train = next(train_gen) # loading the first batch to create X_train and y_train
for i in range(len(train_gen)-1): # -1 since we have already loaded the first batch. 
    img, label = next(train_gen)
    X_train = np.append(X_train, img, axis=0 )
    y_train = np.append(y_train, label, axis=0)
print(X_train.shape, y_train.shape)

In [ ]:
class_labels = np.argmax(y_train, axis=1) # un-doing One Hot encoding
data = pd.DataFrame(class_labels, columns = ['Images']) # defining a dataframe for later use

plt.figure(figsize = (12, 6)) # plotting the percentrage of each Species in the dataset 
#p = sns.countplot(x=data['Images'])
percentage = lambda i: len(i) / float(len(data['Images'])) * 100

ax = sns.barplot(x=data['Images'], y=data['Images'],  estimator=percentage, palette =sns.color_palette("GnBu", 8))
ax.set(ylabel="Percent")
ax.set(xlabel="Species")

ax.set(title='Percentage of each Species in the dataset')
plt.show()

From the plot we can clearly see that Species1 and 6 are under represented! 
Let's have a closer look at the number of images in each class

In [ ]:
data.value_counts(ascending = True)

Only Species1 and 6 have less than 500 images! This is one problem that must be handle. 

To visualise each image we can also re-use the code from the lab. 
(This is Lomurno's code, all thanks to him!) 

In [ ]:
# Lomurno's code! 
def get_next_batch(generator):
    batch = next(generator)

    image = batch[0]
    target = batch[1]

    print("(Input) image shape:", image.shape)
    print("Target shape:",target.shape)

  # Visualize only the first sample
    image = image[0]
    target = target[0]
    target_idx = np.argmax(target)
    print()
    print("Categorical label:", target)
    print("Label:", target_idx)
    print("Class name:", labels[target_idx])
    fig = plt.figure(figsize=(6, 4))
    plt.imshow(np.uint8(image))

    return batch

In [ ]:
# Get a sample from dataset and show info
# Lomurno's code!
_ = get_next_batch(train_gen)

This function will be very useful to visualise the images after the preprocessing step, thus having an additional control. 

# Our Best Model
## ResNet-50 fine tuned with augmentations <br> (a.k.a. ResNet_On_Drugs) 

In [ ]:
#ratio of training partition
ratio = 0.8

#Dataset dir
dataset_dir =  Path(PATH)

#Directory to put processed data
training_dir = Path(TRAINING_PATH) 
validation_dir = Path(VALIDATION_PATH)

#Create non-exist directory
training_dir.mkdir(exist_ok=True, parents=True)
validation_dir.mkdir(exist_ok=True, parents=True)
for i in range (1,9):
    (training_dir/f'Species{i}').mkdir(exist_ok=True, parents=True)
    (validation_dir/f'Species{i}').mkdir(exist_ok=True, parents=True)

#Splitting (I suggest to set the seed for randomly shuffling, for this case is set to 42)
for class_folder in tqdm(list(dataset_dir.glob('*')),desc='Classes'):
    imgs = list(class_folder.glob('*'))
    random.shuffle(imgs) 
    for idx,img in enumerate(imgs):
        if (idx+1)<=(len(imgs)*ratio):
            shutil.copy(img, Path(str(img).replace(str(dataset_dir),str(training_dir))))
        else:
            shutil.copy(img, Path(str(img).replace(str(dataset_dir),str(validation_dir))))

In [ ]:
#Quick check to very if the stratification worked
print(len(list((training_dir/'Species2').glob('*')))/len(list((validation_dir/'Species2').glob('*'))))

## Meta data of the model

In [ ]:
#meta data
input_shape = (96, 96, 3)

# image data 
target_size = (96, 96)
color_mode = 'rgb'
classes = None
class_mode = 'categorical'
batch_size = 16 

## Defining the generators for the model

In [ ]:
from tensorflow.keras.applications.resnet import preprocess_input 

# Generator with augmentations and pre-processing 
train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                        height_shift_range=30,
                                        width_shift_range=30,
                                        horizontal_flip=True,
                                        vertical_flip = True,
                                        fill_mode='reflect') 

# Generator without augmentation, but with pre-processing 
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input) 

#  Defining the training and validation generators! 
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               shuffle=True,
                                               target_size=target_size,
                                               color_mode=color_mode,
                                               classes=classes, 
                                               class_mode=class_mode,
                                               batch_size=batch_size,
                                               seed=seed)

valid_gen = valid_data_gen.flow_from_directory(directory=validation_dir, 
                                               shuffle=False,             
                                               target_size=target_size,
                                               color_mode=color_mode,
                                               classes=classes, 
                                               class_mode=class_mode,
                                               batch_size=batch_size,
                                               seed=seed)

## Importing ResNet50

In [ ]:
# importing our backbone of choice, ResNet50
supernet2 = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(96, 96, 3)
)

supernet2.trainable = False # freezing the ResNet50 layers

## Building the model

In [ ]:
inputs = tfk.Input(shape=(96,96,3))
x = supernet2(inputs,training=False)
x = tfkl.GlobalAveragePooling2D(name='gpooling')(x)
outputs = tfkl.Dense(8, activation='softmax', kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

# Connect input and output through the Model class
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
tl_model.summary()

## Training the model

In [ ]:
history = tl_model.fit(
    x = train_gen,
    epochs = 25,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

# for the best model, early stop did not triggered. So we avoid using early stop and train for all the 25 epochs

In [ ]:
tl_model.save('ResNet50_vanilla') # just to be sure we save the model! 

## Using a bigger batch size

In [ ]:
batch_size = 64 # using a bigger batch size

aug_train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                                       shuffle=True,
                                                       target_size=target_size,
                                                       color_mode=color_mode,
                                                       classes=classes, 
                                                       class_mode=class_mode,
                                                       batch_size=batch_size,
                                                       seed=seed)


valid_gen = valid_data_gen.flow_from_directory(directory=validation_dir,
                                               shuffle=False,             
                                               target_size=target_size,
                                               color_mode=color_mode,
                                               classes=classes, 
                                               class_mode=class_mode,
                                               batch_size=batch_size,
                                               seed=seed)

In [ ]:
ft_model = tfk.models.load_model('ResNet50_vanilla') # loading the model and assigning it to a different variable name for later comparison

## Unfreezing all the layers

In [ ]:
selected_model = 'resnet50'

# unfreezing the ResNet50 layers
ft_model.get_layer(selected_model).trainable = True

In [ ]:
# re-compiling the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

## Fine tuning the model

In [ ]:
# WARNING the training will take around 30 minutes to run and trigger early stop! 

history = ft_model.fit(
    x = train_gen,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True)]
).history

# early stop triggered at 116 epochs, with a max validation accuracy of 0.8948. 

In [ ]:
ft_model.save('ResNet-On-Drugs')

## Evaluating the model

In [ ]:
# Plot the training vs validation to search for overfitting 
plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Training', alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Loss')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e', linestyle='--')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

Some interesting comment about the train-validation gap. 

In [ ]:
evaluated_model = ft_model
valid_samples = 713

Y_pred = evaluated_model.predict_generator(valid_gen, valid_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(valid_gen.classes, y_pred))
print('Classification Report')
target_names = labels
print(classification_report(valid_gen.classes, y_pred, target_names=target_names.values()))